This notebook estimates the number of V100 GPU-days used to train our cyclic adversary.

In [5]:
import datetime
import glob
import os
import pprint
import re
from typing import Optional

import dateutil.parser
import numpy as np

HOURS_TO_SECONDS = 60 * 60
DAYS_TO_SECONDS = 24 * 60 * 60
# Estimate of how many V100 GPU-days is 1 A6000 GPU-day.
# Note(tomtseng): I estimated this value using this data:
# https://www.notion.so/chaiberkeley/Benchmark-KataGo-victimplay-on-different-hardware-295ac2d430404430b8a1e0232f72b240
# - On cp505, 3 A6000 victimplay workers generated 250k rows in 9h30m and 7
#   V100 workers generated 250k rows in 6h44m. This corresponds to an A6000 worker
#   being 1.6538x faster.
# - On cp127, 3 A6000 workers generated 1002027 rows in 16h03 and 7 V100 workers
#   generated 1019074 rows in 12h16. This corresponds to an A6000 worker being
#   1.7535x faster.
# - The mean of 1.6537 and 1.7535 is 1.7036.
# As a comparison, at https://lambdalabs.com/gpu-benchmarks with fp16 precision,
# Lambda measures the conversion factor to be 1.48.
A6000_TO_V100_GPU_DAY = 1.703643

## Conversions to A6000 GPU-days

The cyclic adversary was trained using A6000, A4000, A100 40GB,
and A100 80GB GPUs. We'll estimate how many A6000 GPU-days is
1 GPU-day on each of these GPU types.

In [2]:
def get_victimplay_log_line_timestamp(line: str) -> Optional[datetime.datetime]:
    """Fetches the timestamp from a line in a victimplay log file."""
    match = re.match(r"([0-9 \-+:]+): ", line)
    if match is None:
        return None
    return dateutil.parser.parse(match.group(1))


def get_victimplay_duration(log_filename: str) -> Optional[float]:
    """Returns how long the victimplay worker was generating games.

    Args:
      log_filename: Filename of the victimplay worker's log file.

    Returns:
        How long the worker was generating games in seconds.
        Returns None if the worker did not generate any games.
    """
    GAME_START_REGEX = re.compile(r"starting game")
    TIMESTAMP_REGEX = re.compile(r"([0-9 \-+:]+): ")
    with open(log_filename, "r") as f:
        first_game_start_timestamp = None
        for line in f:
            if GAME_START_REGEX.search(line) is not None:
                first_game_start_timestamp = get_victimplay_log_line_timestamp(line)
                assert first_game_start_timestamp is not None
                break
        if first_game_start_timestamp is None:
            return None

        for line in f:
            pass
        last_line = line
        last_timestamp = get_victimplay_log_line_timestamp(last_line)
        assert last_timestamp is not None
        return (last_timestamp - first_game_start_timestamp).total_seconds()


def get_victimplay_games_finished(log_filename: str, game_limit=None) -> int:
    """Returns the number of games the victimplay worker completed.

    Args:
        log_filename: Filename of the victimplay worker's log file.
        game_limit: If not None, then stop counting and return early if
          this number of games is reached.
    """
    GAME_FINISH_REGEX = re.compile("Game #")
    num_games_finished = 0
    with open(log_filename, "r") as f:
        for line in f:
            if GAME_FINISH_REGEX.search(line) is not None:
                num_games_finished += 1
                if game_limit is not None and num_games_finished >= game_limit:
                    break
    return num_games_finished


def get_victimplay_gpu_type(log_filename: str) -> Optional[str]:
    """Parses GPU type from victimplay log."""
    GPU_TYPE_REGEX = re.compile(r"Found GPU (.*) memory")
    with open(log_filename, "r") as f:
        for line in f:
            match = GPU_TYPE_REGEX.search(line)
            if match is not None:
                return match.group(1)
    return None

In [3]:
# Victimplay runs on different GPU types with the same adversary+victim setup.
# Experiment link: https://www.notion.so/chaiberkeley/adv-s300mil-v600-vs-cp505-v8-benchmarking-by-GPU-6b151352b64241d2a67c21b6fb1c8da6
benchmarking_directories = [
    "/nas/ucb/k8/go-attack/victimplay/ttseng-bench-a100-40gb-20230203-163919",
    "/nas/ucb/k8/go-attack/victimplay/ttseng-bench-a100-80gb-20230203-163757",
    "/nas/ucb/k8/go-attack/victimplay/ttseng-bench-a4000-20230203-163607",
    "/nas/ucb/k8/go-attack/victimplay/ttseng-bench-a6000-20230203-163319",
    "/nas/ucb/k8/go-attack/victimplay/ttseng-bench-a100-40gb-20230206-131111",
    "/nas/ucb/k8/go-attack/victimplay/ttseng-bench-a100-80gb-20230206-124106",
    "/nas/ucb/k8/go-attack/victimplay/ttseng-bench-a4000-20230206-124105",
    "/nas/ucb/k8/go-attack/victimplay/ttseng-bench-a6000-20230206-121545",
]

# We'll count the number of games each victimplay worker completed
# as a proxy for how performant the GPU type is for our workload.

# Key = GPU type
# Value = List of number of victimplay games per hour
#        (1 entry for each victimplay worker)
gpu_type_to_game_throughputs = {}
for directory in benchmarking_directories:
    victimplay_logfiles = glob.glob(f"{directory}/selfplay/log*.log")
    for logfile in victimplay_logfiles:
        duration = get_victimplay_duration(logfile)
        if duration is None:
            continue
        gpu_type = get_victimplay_gpu_type(logfile)
        assert gpu_type is not None
        games_finished = get_victimplay_games_finished(logfile)

        throughputs = gpu_type_to_game_throughputs.get(gpu_type, [])
        throughputs.append(games_finished / (duration / HOURS_TO_SECONDS))
        gpu_type_to_game_throughputs[gpu_type] = throughputs

gpu_type_to_avg_throughput = {}
for gpu_type, throughputs in gpu_type_to_game_throughputs.items():
    avg = np.mean(throughputs)
    stddev = np.std(throughputs)
    print(f"{gpu_type} throughput: Avg. {avg} +- {stddev}")
    gpu_type_to_avg_throughput[gpu_type] = avg

a6000_throughput = gpu_type_to_avg_throughput["NVIDIA RTX A6000"]
# Key = GPU type
# Value = Estimate of how many A6000 GPU-days is 1 GPU-day
#         of this GPU type.
gpu_type_to_a6000_gpu_days = {
    gpu_type: throughput / a6000_throughput
    for gpu_type, throughput in gpu_type_to_avg_throughput.items()
}

print("Conversions to A6000 GPU-days:")
pprint.pprint(gpu_type_to_a6000_gpu_days)

NVIDIA A100-SXM4-40GB throughput: Avg. 180.94829353581846 +- 7.091347343534531
NVIDIA A100-SXM4-80GB throughput: Avg. 202.9935724100512 +- 1.4472585304743872
NVIDIA RTX A4000 throughput: Avg. 67.93128417907852 +- 2.071138168948876
NVIDIA RTX A6000 throughput: Avg. 108.38931872111866 +- 1.2438836439931913
Conversions to A6000 GPU-days:
{'NVIDIA A100-SXM4-40GB': 1.6694291990282837,
 'NVIDIA A100-SXM4-80GB': 1.8728189715108872,
 'NVIDIA RTX A4000': 0.6267341190128057,
 'NVIDIA RTX A6000': 1.0}


## V100 GPU-days used for training cyclic-adversary

In [10]:
def get_modified_time(filename: str) -> datetime.datetime:
    """Returns the modified time of a file."""
    epoch_time = os.path.getmtime(filename)
    return datetime.datetime.utcfromtimestamp(epoch_time)


ADVERSARY_FILE = "/nas/ucb/k8/go-attack/victimplay/ttseng-avoid-pass-alive-coldstart-39-20221025-175949/models/t0-s545065216-d136760487/model.bin.gz"
adversary_written_time = get_modified_time(ADVERSARY_FILE)

# Key = GPU type
# Value = how many GPU-seconds did we use of that GPU during victimplay
gpu_type_to_seconds = {}

victimplay_logfiles = glob.glob(
    "/nas/ucb/k8/go-attack/victimplay/ttseng-avoid-pass-alive-coldstart-39-20221025-175949/selfplay/log*.log"
)
for logfile in victimplay_logfiles:
    finished_a_game = get_victimplay_games_finished(logfile, game_limit=1) > 0
    if not finished_a_game:
        # This victimplay worker didn't generate any data, so we'll just skip
        # it. (In particular, we had a lot of workers crash early and not
        # generate any data on Nov. 1 due to a bug.)
        continue

    # Parse creation time (in UTC) from filename.
    creation_time_match = re.search(r"log(\d{8}-\d{6})-", logfile)
    assert creation_time_match is not None
    creation_time = dateutil.parser.parse(creation_time_match.group(1))
    if creation_time > adversary_written_time:
        continue

    modified_time = min(get_modified_time(logfile), adversary_written_time)
    time_alive_seconds = (modified_time - creation_time).total_seconds()
    gpu_type = get_victimplay_gpu_type(logfile)
    assert gpu_type is not None
    gpu_type_to_seconds[gpu_type] = (
        gpu_type_to_seconds.get(gpu_type, 0) + time_alive_seconds
    )

total_v100_days = 0
for gpu_type, seconds in gpu_type_to_seconds.items():
    days = seconds / DAYS_TO_SECONDS
    print(f"GPU {gpu_type}: {seconds / DAYS_TO_SECONDS} GPU-days")
    total_v100_days += (
        days * gpu_type_to_a6000_gpu_days[gpu_type] * A6000_TO_V100_GPU_DAY
    )
print("Total V100 GPU-days:", total_v100_days)

GPU NVIDIA RTX A6000: 347.5863040065741 GPU-days
GPU NVIDIA A100-SXM4-40GB: 299.65091498769675 GPU-days
GPU NVIDIA A100-SXM4-80GB: 222.4628226519676 GPU-days
GPU NVIDIA RTX A4000: 61.84087257660878 GPU-days
Total V100 GPU-days: 2220.2261800630413
